In [101]:
import pandas as pd
import datetime
import os
import numpy as np

In [94]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post'

In [95]:
store_list=pd.read_excel('/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/SOTF Wave 1 Store IDs_8.20.xlsx',
                         dtype=str,sheetname="Stores_6_8")
store_set=set(store_list['SOTF'].unique().tolist())

In [96]:
store_wide_sales=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/wide_sales_date2018-09-01.xlsx",
                               dtype=str,sheetname="sales")

In [97]:
store_wide_trans=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/wide_sales_date2018-09-01.xlsx",
                               dtype=str,sheetname="transactions")

In [98]:
First_week_2018=datetime.date(2018,5,19)
First_week_2017=datetime.date(2017,5,20)

weeks_2018=[str(First_week_2018+datetime.timedelta(days=i*7)) for i in range(8)]
weeks_2017=[str(First_week_2017+datetime.timedelta(days=i*7)) for i in range(8)]

In [99]:
SOTF_sales=store_wide_sales[['location_id']+weeks_2017+weeks_2018]
SOTF_sales=SOTF_sales[SOTF_sales['location_id'].isin(store_set)]
SOTF_trans=store_wide_trans[['location_id']+weeks_2017+weeks_2018]
SOTF_trans=SOTF_trans[SOTF_trans['location_id'].isin(store_set)]


In [102]:
for col in SOTF_trans.columns.tolist()[1:]:
    SOTF_trans[col]=SOTF_trans[col].astype(int)
    
for col in SOTF_sales.columns.tolist()[1:]:
    SOTF_sales[col]=SOTF_sales[col].apply(lambda x: np.round(float(x),2))

# Rewards

In [9]:
start_2017=datetime.datetime.strptime(weeks_2017[0],"%Y-%m-%d").date()-datetime.timedelta(days=6)
start_2018=datetime.datetime.strptime(weeks_2018[0],"%Y-%m-%d").date()-datetime.timedelta(days=6)
end_2017=datetime.datetime.strptime(weeks_2017[7],"%Y-%m-%d").date()
end_2018=datetime.datetime.strptime(weeks_2018[7],"%Y-%m-%d").date()

In [19]:
data_from_SP=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/From_Sp/combinedtransactions_0811.csv",
                        dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_id','location_id','total_transaction_amt'])
data_from_SP['transaction_date']=data_from_SP['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
data_from_SP=data_from_SP[(data_from_SP['transaction_date']>=start_2017) & (data_from_SP['transaction_date']<=end_2017) |\
                         (data_from_SP['transaction_date']>=start_2018) & (data_from_SP['transaction_date']<=end_2018)]
data_from_SP=data_from_SP.drop_duplicates()

In [45]:
def week_end_func(x):
    if x.weekday()==6:
        week=x+datetime.timedelta(days=6)
    else:
        diff=x.weekday()
        week=x+datetime.timedelta(days=5-diff)
    return week
        

In [52]:
data_from_SP['week_end_date']=data_from_SP['transaction_date'].apply(lambda x: week_end_func(x))
data_from_SP=data_from_SP[data_from_SP['location_id'].isin(store_set)]
data_from_SP['total_transaction_amt']=data_from_SP['total_transaction_amt'].astype(float)

In [105]:
rewards_by_week_sales=data_from_SP.groupby(['location_id','week_end_date'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_by_week_trans=data_from_SP.groupby(['location_id','week_end_date'])['total_transaction_amt'].count().to_frame().reset_index()


# Combine

In [106]:
output=store_list.copy()
output=output.rename(columns={"SOTF":"location_id"})

In [107]:
output_store_total_sales=pd.merge(output,SOTF_sales,on="location_id",how="left")
output_store_total_trans=pd.merge(output,SOTF_trans,on="location_id",how="left")

In [108]:
rewards_by_week_trans=rewards_by_week_trans.pivot(index="location_id",columns="week_end_date",values="total_transaction_amt").reset_index()
rewards_by_week_sales=rewards_by_week_sales.pivot(index="location_id",columns="week_end_date",values="total_transaction_amt").reset_index()


In [109]:
output_store_rewards_sales=pd.merge(output,rewards_by_week_sales,on="location_id",how="left")
output_store_rewards_trans=pd.merge(output,rewards_by_week_trans,on="location_id",how="left")

In [112]:
output_store_rewards_sales=output_store_rewards_sales.fillna(0)
output_store_rewards_trans=output_store_rewards_trans.fillna(0)

In [113]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/"+"BL_35_stores_opened_20180608_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

In [114]:
output_store_total_sales.to_excel(writer,"store_total_sales",index=False)
output_store_total_trans.to_excel(writer,"store_total_trans",index=False)
output_store_rewards_sales.to_excel(writer,"store_rewards_sales",index=False)
output_store_rewards_trans.to_excel(writer,"store_rewards_trans",index=False)
writer.save()